In [ ]:
import sys
import math
import wandb
import torch
sys.path.append('..')
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from utils.data_modules.contrastive import EEGContrastiveDataModule
from models.trainers.contrastive import ContrastiveTrainerModel, PlottingCallback

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


S:\PolySecLabProjects\eeg-image-decoding\data\all-joined-1\eeg\epochs


In [2]:
dm = EEGContrastiveDataModule(
    input_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=32, 
    num_workers=4
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 1
subject = 1
session = 1
num_channels = sample_data['input']['num_channels']
timesteps = sample_data['input']['num_timesteps']
num_fine_labels = sample_data['output']['fine_labels_shape']

Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320
Original dataframe size: 640


In [4]:
lightning_model = ContrastiveTrainerModel(num_channels, timesteps, num_fine_labels)
    
checkpoint_callback = ModelCheckpoint(
    dirpath="S:\\PolySecLabProjects\\eeg-image-decoding\\updated-code\\models\\check_points\\contrastive_encoder",
    filename=f"subj{subject}_session{session}_{{epoch}}",
    every_n_epochs=5,
    save_top_k=-1
)

plotting_callback = PlottingCallback()

logger = pl.loggers.WandbLogger(project="contrastive_eeg_net")

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[checkpoint_callback, plotting_callback],
    logger=logger,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.fit(lightning_model, dm)

if logger and isinstance(logger, pl.loggers.WandbLogger):
    logger.finalize('success')
    wandb.finish()

s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\braindecode\models\base.py:23: UserWarning: EEGNetv4: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\braindecode\models\base.py:23: UserWarning: EEGNetv4: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\braindecode\models\base.py:23: UserWarning: EEGNetv4: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32

Creating Datasets...
Original dataframe size: 2879
Original dataframe size: 320


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                      | Params | Mode 
-----------------------------------------------------------------------------
0  | encoder               | EEGEncoderDefiner         | 859 K  | train
1  | train_loss            | MeanMetric                | 0      | train
2  | train_accuracy        | MultilabelAccuracy        | 0      | train
3  | val_loss              | MeanMetric                | 0      | train
4  | val_accuracy          | MultilabelAccuracy        | 0      | train
5  | v2_acc                | MultilabelAccuracy        | 0      | train
6  | v4_acc                | MultilabelAccuracy        | 0      | train
7  | v10_acc               | MultilabelAccuracy        | 0      | train
8  | v80_acc               | MultilabelAccuracy        | 0      | train
9  | top5_acc              | MeanMetric                | 0      | train
10 | test_precision        | MultilabelPrecision       | 0      | train
11 | test_recal

Original dataframe size: 640


s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
s:\PolySecLabProjects\eeg-image-decoding\env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MultilabelAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


Epoch 0: 100%|██████████| 90/90 [03:35<00:00,  0.42it/s, v_num=epjk, val_loss=3.470, val_accuracy=0.0339]

`Trainer.fit` stopped: `max_epochs=1` reached.


No test predictions available for confusion matrix visualization
Epoch 0: 100%|██████████| 90/90 [03:36<00:00,  0.42it/s, v_num=epjk, val_loss=3.470, val_accuracy=0.0339]


epoch,▁▁
top5_acc,▁
train_accuracy,▁
train_loss,▁
trainer/global_step,▁▁▁▁▁▁▁▁▁
v10_acc,▁
v2_acc,▁
v4_acc,▁
v80_acc,▁
val_accuracy,▁
val_loss,▁


In [5]:
results = []
for epoch in range(epochs):
    if epoch < len(plotting_callback.train_losses):
        epoch_results = {
            "epoch": epoch + 1,
            "train_loss": plotting_callback.train_losses[epoch],
            "train_accuracy": plotting_callback.train_accuracies[epoch],
            "test_loss": plotting_callback.test_losses[epoch],
            "test_accuracy": plotting_callback.test_accuracies[epoch],
            "v2_acc": plotting_callback.v2_accs[epoch],
            "v4_acc": plotting_callback.v4_accs[epoch],
            "v10_acc": plotting_callback.v10_accs[epoch],
            "v80_acc": plotting_callback.v80_accs[epoch]
        }
        results.append(epoch_results)

results

[{'epoch': 1,
  'train_loss': 0,
  'train_accuracy': 0,
  'test_loss': 0,
  'test_accuracy': 0,
  'v2_acc': 0,
  'v4_acc': 0,
  'v10_acc': 0,
  'v80_acc': 0}]